In [9]:
import pandas as pd
import numpy as np
import sqlite3
from sklearn.impute import KNNImputer

In [2]:
# Imputation function using limited forward fill
def impute_with_limit(series):
   last_valid_index = series.last_valid_index()
   
   if series[last_valid_index] == np.nan:
      # If there are no valid values, return the original series
      return series
   else:
      return series[:last_valid_index + 1].ffill()

In [3]:
# Connect to the SQLite database
conn = sqlite3.connect('final_data.db')

# Replace 'your_table' with the actual table name and 'conn' with your database connection
query = 'SELECT * FROM features_complete'

# Use the pandas read_sql function to read the data into a DataFrame
events_df = pd.read_sql(query, conn)

# Close the database connection
conn.close()

# Now, you can work with the DataFrame 'df'
events_df['date'] = pd.to_datetime(events_df['date'])
events_df = events_df.sort_values(by='date')
stock_df = pd.read_csv('stock_data.zip', compression='zip', header=[0, 1], index_col=0, parse_dates=True)
events_df


,ticker,disease,stage,date,catalyst,NCAR,avg_5_10,avg_5_30,std_5_10,std_5_30,mkt_cap,roe,op_margin,net_margin,debt_ratio,20_day_pct_ret,bio_score,sec_score
0,PCRX,Postsurgical pain,Approved,2011-10-31,"Approved October 31, 2011.",-1.041054,0.370303,0.369892,-0.168146,-0.197195,4.356451e+08,0.008398,0.067019,0.010490,2.183010,-28.118609,0.633965,0.097330
1,INCY,Myelofibrosis,Approved,2011-11-16,"Approved November 16, 2011.",0.754430,-0.297576,-1.400215,-0.212675,-0.971217,2.858325e+09,0.086178,0.139470,0.117756,4.385141,3.709428,0.878714,0.067742
2,HALO,Overactive bladder,Approved,2011-12-08,"Approved December 8, 2011.",3.427405,0.291818,0.733871,-0.396335,-0.382197,1.283978e+09,1.018970,0.398417,0.325227,1.151618,2.302632,0.044723,0.126709
3,CRIS,Basal Cell Carcinoma,Approved,2012-01-30,"Approved January 30, 2012.",-2.355933,13.045455,39.500000,-2.882373,-11.948936,4.351435e+08,-0.243362,-3.540007,-3.924143,1.747271,-2.736842,0.516248,0.039100
4,CORT,Cushing's Syndrome,Approved,2012-02-17,"Approved February 17, 2012.",-0.955556,-0.210152,-0.388011,-0.144318,-0.129160,3.498192e+08,0.033080,0.218318,0.161087,7.150929,31.683168,0.115455,0.088730
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2288,VSTM,KRAS mutant advanced solid tumors,Phase 1/2,2023-09-28,Phase 1/2 data reported an overall response ra...,-0.796273,-0.271061,-0.646075,-0.001864,0.007625,2.100859e+08,-0.087558,0.000000,0.000000,37.819699,-18.870192,0.946592,0.071477
2289,FOLD,Pompe Disease,Approved,2023-09-28,"FDA Approved on September 28, 2023.",-1.183925,-0.282273,-0.473710,-0.113667,-0.029432,3.405254e+09,-0.292714,-2.314387,-1.538544,1.736064,-8.431703,0.945842,0.118861
2286,MRK,Pulmonary arterial hypertension (PAH),PDUFA,2023-09-28,"PDUFA date on March 26, 2024.",-0.640558,-1.012121,-2.636022,-0.343216,-0.690639,2.646635e+11,-0.008802,0.000000,-0.041344,2.159854,1.198466,0.853895,0.286249
2287,IMGN,Cancer - ovarian cancer and relapsed endometri...,Phase 3,2023-09-28,Additional Phase 3 data reported at ESGO noted...,-0.427396,-0.173788,-0.527688,-0.026306,0.010328,3.983095e+09,-0.110565,-1.677301,-1.674281,2.241927,-7.562500,0.116138,0.122695


In [49]:
test = stock_df['RHHBY']
test

,open,high,low,close,volume,vwap,transactions
2008-01-02 05:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-03 05:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-04 05:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-07 05:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-08 05:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2023-09-25 04:00:00,34.02,34.2699,33.95,34.13,1453975.0,34.0827,1304.0
2023-09-26 04:00:00,33.79,34.1300,33.70,34.03,2099048.0,34.0151,5224.0
2023-09-27 04:00:00,34.58,34.6100,34.09,34.20,1593246.0,34.2349,3484.0
2023-09-28 04:00:00,33.88,34.1200,33.82,33.95,2353980.0,33.9493,5308.0


In [1]:
from importlib import reload, import_module
import yfinance as yf
from ncar import *

In [33]:
# Function to find the closest available index in the stock data
def find_closest_index(date, available_dates):
   closest_index = min(available_dates, key=lambda x: abs(pd.Timestamp(x) - pd.Timestamp(date)))
   return (available_dates == closest_index).idxmax()

# Function to get the returns based on the window built for applying to a dataframe
def get_returns(row, stock_df, window, lag):
   available_dates = pd.Series(stock_df.index.date)
   idx = find_closest_index(row['date'], available_dates)
   close_prices = stock_df[row['ticker']].iloc[idx+lag:idx+lag+window, stock_df[row['ticker']].columns.get_loc('close')]
   return (close_prices[-1] - close_prices[0])/close_prices[0]

# Function to get the volatility based on the window built for applying to dataframe
def get_vol(row, stock_df, window, lag):
   available_dates = pd.Series(stock_df.index.date)
   idx = find_closest_index(row['date'], available_dates)
   close_prices = stock_df[row['ticker']].iloc[idx:idx+window, stock_df[row['ticker']].columns.get_loc('close')]
   return close_prices.pct_change().std()


In [34]:
returns = events_df.apply(get_returns, stock_df=stock_df, window=20, lag=0, axis=1)
vol = events_df.apply(get_vol, stock_df=stock_df, window=20, lag=0, axis=1)

In [11]:
events_df['RET_20'] = returns.reset_index(drop=True)
events_df['VOL_20'] = vol.reset_index(drop=True)


In [12]:
import pandas as pd
import sqlite3

# Create a Pandas DataFrame (replace this with your actual DataFrame)
# SQLite database file path
db_file_path = 'final_data.db'

# SQLite connection
conn = sqlite3.connect(db_file_path)

# Use the to_sql method to write the DataFrame to a new table in the SQLite database
# if_exists='replace' will replace the table if it already exists, you can use 'fail' or 'append' if needed
events_df.to_sql(name='ALL_FEATURES', con=conn, index=False, if_exists='replace')

# Close the connection
conn.close()
